Step 1: Load the data and prepare the input data for Meridian

In [6]:
# Notebook-wide display settings for GitHub
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # crisp PNGs in the .ipynb
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
# Suppress TensorFlow/XLA messages - MUST be set before importing TensorFlow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress all TF messages
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations messages
os.environ['XLA_FLAGS'] = '--xla_hlo_profile=false'  # Suppress XLA messages

# Additional TensorFlow warning suppression
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)


In [7]:
# Step 1: Load the data and prepare the input data for Meridian
from meridian.data import data_frame_input_data_builder
import pandas as pd

# Use relative path (your notebook lives in marketing_mix_model/model/)
data_path = '../data/mmm_daily_agg.csv'
mm = pd.read_csv(data_path, parse_dates=['click_date']) \
       .rename(columns={
           'clicks_google': 'clicks_Google',
           'clicks_meta': 'clicks_Meta',
           'clicks_linkedin': 'clicks_LinkedIn',
           'spend_google': 'spend_Google',
           'spend_meta': 'spend_Meta',
           'spend_linkedin': 'spend_LinkedIn',
           'click_date': 'time'
       }) \
       .sort_values('time')

from meridian.data import data_frame_input_data_builder as dfib

df_builder = dfib.DataFrameInputDataBuilder(
    kpi_type='revenue',
    default_kpi_column='revenue_total'
)

builder = df_builder.with_kpi(mm, time_col='time')

control_columns = ['holiday_flag', 'promo_discount', 'competitor_index']
builder = builder.with_controls(mm, control_cols=control_columns, time_col='time')

channels = ['Google','Meta','LinkedIn']
media_columns       = [f'clicks_{ch}' for ch in channels]
media_spend_columns = [f'spend_{ch}'  for ch in channels]

builder = builder.with_media(
    mm,
    media_cols=media_columns,
    media_spend_cols=media_spend_columns,
    media_channels=channels
)

input_data = builder.build()
print("Input data ready.")

Input data ready.


In [8]:
# Step 2: Specify the model configuration
import tensorflow_probability as tfp
import tensorflow as tf
from meridian.model import prior_distribution, spec, model

tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')

roi_mu, roi_sigma = 0.2, 0.9
priors = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name='roi_m')
)

model_spec = spec.ModelSpec(
    prior=priors,
    max_lag=14,
    hill_before_adstock=False,
    knots=10
)

mmm = model.Meridian(input_data=input_data, model_spec=model_spec)
print("Model initialized.")

Model initialized.


Step 3: Sample the model

In [9]:
# Step 3: Sample the model
mmm.sample_prior(300)

mmm.sample_posterior(
    n_chains=4,
    n_adapt=2000,
    n_burnin=1000,
    n_keep=1500,
    seed=0
)
print("Sampling done.")

W0000 00:00:1755155841.280586   43200 assert_op.cc:38] Ignoring Assert operator mcmc_retry_init/assert_equal_1/Assert/AssertGuard/Assert


Sampling done.


Step 4: Model diagnostics

4.1 Convergence diagnostics

In [21]:
# Step 4: Model diagnostics (ensure figures are emitted to the notebook)
from meridian.analysis import visualizer

diagnostics = visualizer.ModelDiagnostics(mmm)

# Let this be the last line in the cell so the output is embedded in the notebook
diagnostics.plot_rhat_boxplot()

alt.LayerChart(...)

We ran an r-hat diagnostic to check convergence of the model. The r-hat value should be close to 1.0 for all parameters, indicating that the chains have converged. A common rule of thumb is < 1.10 for acceptable results. Our results are all extremely close to 1.0 which is to be expected considering that we used perfect dummy-data to train the model. This is an indicator that the model has learned the underlying patterns in the data effectively and we can make decisions based on it.

4.2 Model Fit

In [20]:
# 4.2 Model Fit
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()  # leave this as the last line in the cell

alt.LayerChart(...)

The model replicates the overall trend in the data, capturing the peaks and troughs in revenue. The model fit is good, indicating that the model has learned the underlying patterns in the data effectively.

Step 5: Analyze the results

5.1 Channel Contribution Analysis

In [22]:
media_summary = visualizer.MediaSummary(mmm)

# Contribution of media channels
media_summary.plot_channel_contribution_area_chart()

alt.Chart(...)

In [23]:
media_summary.plot_channel_contribution_bump_chart()

alt.Chart(...)

In [24]:
media_summary.plot_contribution_waterfall_chart()

alt.LayerChart(...)

In [25]:
media_summary.plot_contribution_pie_chart()

alt.LayerChart(...)

5.2 Spend vs Contribution to ROI

In [26]:
media_summary.plot_spend_vs_contribution()

alt.FacetChart(...)

In [27]:
media_summary.plot_roi_bar_chart()

alt.LayerChart(...)

5.3 Response and Saturation Curves

These curves show the relationship between media spend and the response (revenue) generated by each channel. The response curve shows how much revenue is generated for each dollar spent on media, while the saturation curve shows how much additional revenue is generated as spend increases. The curves can help identify the optimal level of spend for each channel to maximize ROI.

In [28]:
media_effects = visualizer.MediaEffects(mmm)

media_effects.plot_response_curves()

alt.FacetChart(...)

In [29]:
media_effects.plot_adstock_decay()

alt.FacetChart(...)

In [30]:
hill_chart = media_effects.plot_hill_curves()
hill_chart['media']

alt.FacetChart(...)

Step 6: Create a Summary Report

In [ ]:
from meridian.analysis import summarizer

summary = summarizer.Summarizer(mmm)

summary.output_model_results_summary(
    filename='mmm_summary.html',
    filepath='//marketing_mix_model/results/mmm_summary.html',
)